In [2]:
"""Sanity check on booth function. All optimizers should converge."""
import pytest
import torch
import numpy as np
import torchzero as tz

def booth(x,y):
    return (x + 2 * y - 7) ** 2 + (2 * x + y - 5) ** 2

x0 = (0, -8)

__pylance = torch.tensor(float('inf'))
PRINT_LOSSES = True

def _ensure_float(x):
    if isinstance(x,torch.Tensor): return x.detach().cpu().item()
    if isinstance(x, np.ndarray): return x.item()
    return x

def _test_optimizer(lmbda, tol=1e-1, niter=100, allow_non_tensor=False):
    params = torch.tensor(x0, dtype=torch.float32, requires_grad=True)
    opt = lmbda([params])

    def closure(backward=True):
        loss = booth(*params)
        if backward:
            opt.zero_grad()
            loss.backward()
        return loss

    loss = __pylance
    losses = []
    for i in range(niter):
        loss = opt.step(closure)
        losses.append(loss)

        if allow_non_tensor:
            assert isinstance(loss, (torch.Tensor, np.ndarray, int, float)), (opt.__class__.__name__, i, type(loss), loss)
        else:
            assert isinstance(loss, torch.Tensor), (opt.__class__.__name__, i, type(loss), loss)

        if isinstance(loss, torch.Tensor): assert torch.isfinite(loss), (opt.__class__.__name__, i, loss)
        else: assert np.isfinite(loss), (opt.__class__.__name__, i, loss)

    assert loss <= tol, (opt.__class__.__name__, tol, loss, [i.detach().cpu().item() for i in losses])
    if PRINT_LOSSES: print(opt.__class__.__name__, _ensure_float(loss))

In [3]:
import nevergrad as ng
from torchzero.optim.wrappers.nevergrad import NevergradOptimizer
opt = lambda p: tz.optim.Modular(p, tz.m.MinibatchRprop(1e-1))
_test_optimizer(opt, niter=100)

Modular 4.547473508864641e-13
